In [45]:
pip install nbformat

In [46]:
pip show nbformat


Name: nbformat
Version: 5.9.2
Summary: The Jupyter Notebook format
Home-page: 
Author: 
Author-email: Jupyter Development Team <jupyter@googlegroups.com>
License: BSD 3-Clause License

- Copyright (c) 2001-2015, IPython Development Team
- Copyright (c) 2015-, Jupyter Development Team

All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its
   contributors may be used to endorse or promote products derived from
   this software without specific prior written permission.

THIS

In [47]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Defining Graphing Function

In [48]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Question 1: Use yfinance to Extract Stock Data

In [49]:
# Using the Ticker function enter the ticker symbol of the stock we want to extract data on to create a ticker object. 
# The stock is Tesla and its ticker symbol is TSLA

Tesla = yf.Ticker("TSLA")

In [50]:
# Using the ticker object and the function history extract stock information and save it in a dataframe named tesla_data. 
# Set the period parameter to max so we get information for the maximum amount of time.

tesla_data = Tesla.history(period = "max")
tesla_data 

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0
...,...,...,...,...,...,...,...
2023-08-16 00:00:00-04:00,228.020004,233.970001,225.380005,225.600006,112484500,0.0,0.0
2023-08-17 00:00:00-04:00,226.059998,226.740005,218.830002,219.220001,120718400,0.0,0.0
2023-08-18 00:00:00-04:00,214.119995,217.580002,212.360001,215.490005,135813700,0.0,0.0


In [51]:
# Reset the index using the reset_index(inplace=True) function on the tesla_data DataFrame and 
# display the first five rows of the tesla_data dataframe using the head function

tesla_data.reset_index(inplace=True)

In [52]:
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Question 2: Use Webscraping to Extract Tesla Revenue Data

In [53]:
html_data = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"

In [54]:
# Using the request library to download the webpage

data = requests.get(html_data).text
print(data)


<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->
    <head>
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
		<link rel="canonical" href="https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue" />
		<title>Tesla Revenue 2010-2022 | TSLA | MacroTrends</title>
		<meta name="description" content="Tesla annual/quarterly revenue history and growth rate from 2010 to 2022. Revenue can be defined as the amount of money a company receives from its customers in exchange for the sales of goods or services.  Revenue is the top line item on an income statement from which all costs and expenses are subtracted to arrive at net income.    
				
				&lt;ul style='margin-top:10px;'&gt;
				&lt;li&gt;Tesla r

In [55]:
# Parsing the data

soup = BeautifulSoup(data, "html5lib")

In [56]:
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

In [57]:
data_list = []

In [58]:
# Using BeautifulSoup to extract the table with Tesla Revenue and store it into a dataframe named tesla_revenue. 
# The dataframe should have columns Date and Revenue

table = soup.find_all('tbody')[1]
for row in table.find_all('tr'):
    col = row.find_all('td')
    Date = col[0].text
    Revenue = col[1].text
    data_list.append(
    {
        'Date': Date,
        'Revenue': Revenue
    })
df = pd.DataFrame(data_list)
tesla_revenue = pd.concat([tesla_revenue, df], ignore_index = True)
tesla_revenue.head()

,Date,Revenue
0,2022-09-30,"$21,454"
1,2022-06-30,"$16,934"
2,2022-03-31,"$18,756"
3,2021-12-31,"$17,719"
4,2021-09-30,"$13,757"


In [59]:
# Removing the comma from the Revenue table

tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',',"")

In [60]:
# Removing the $ from the revenue table

tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace('$',"")

C:\Users\HP\AppData\Local\Temp\ipykernel_9336\660708832.py:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [61]:
# Execute the following lines to remove an null or empty strings in the Revenue column

tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [62]:
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


## Question 5: Plot Tesla Stock Graph

In [79]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

## Question 3: Use yfinance to Extract Stock Data

In [64]:
GameStop = yf.Ticker("GME")

In [65]:
gme_data = GameStop.history(period = "max")
gme_data 

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0
...,...,...,...,...,...,...,...
2023-08-16 00:00:00-04:00,19.180000,19.340000,18.549999,18.590000,3015600,0.0,0.0
2023-08-17 00:00:00-04:00,18.790001,18.809999,18.299999,18.379999,2056800,0.0,0.0
2023-08-18 00:00:00-04:00,18.180000,18.660000,18.139999,18.360001,2023900,0.0,0.0


In [66]:
# Reset the index using the reset_index(inplace=True) function on the gme_data DataFrame

gme_data.reset_index(inplace=True)

In [67]:
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Question 4: Use Webscraping to Extract GME Revenue Data

In [68]:
html_data = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"

In [69]:
data = requests.get(html_data).text
data

'<!DOCTYPE html>\n<!-- saved from url=(0105)https://web.archive.org/web/20200814131437/https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue -->\n<html class=" js flexbox canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths" style=""><!--<![endif]--><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><script type="text/javascript" async="" src="./GameStop Revenue 2006-2020 _ GME _ MacroTrends_files/analytics.js.download"></script><script async="" type="text/javascript" src="./GameStop Revenue 2006-2020 _ GME _ MacroTrends_files/gpt.js.download"></script><script async="" src=

In [70]:
soup = BeautifulSoup(data, "html5lib")

In [71]:
gme_revenue = pd.DataFrame(columns = ["Date", "Revenue"])

In [72]:
data_list = []

In [73]:
table = soup.find_all('tbody')[1]
for row in table.find_all('tr'):
    col = row.find_all('td')
    Date = col[0].text
    Revenue = col[1].text
    data_list.append(
    {
        'Date': Date,
        'Revenue': Revenue
    })
df = pd.DataFrame(data_list)
gme_revenue = pd.concat([gme_revenue, df], ignore_index = True)
gme_revenue.head()

,Date,Revenue
0,2020-04-30,"$1,021"
1,2020-01-31,"$2,194"
2,2019-10-31,"$1,439"
3,2019-07-31,"$1,286"
4,2019-04-30,"$1,548"


In [74]:
# Removing the comma from the Revenue table

gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',',"")

In [75]:
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace('$',"")

C:\Users\HP\AppData\Local\Temp\ipykernel_9336\1440917798.py:1: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [76]:
# Execute the following lines to remove an null or empty strings in the Revenue column

gme_revenue.dropna(inplace=True)

gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

In [77]:
gme_revenue.tail()

,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


## Question 6: Plot GameStop Stock Graph

In [80]:
make_graph(gme_data, gme_revenue, 'GameStop')